<table align=left width="590" height="144" style="height: 67px; width: 565px;">
<tbody>
<tr>
<td width=82><img src="https://static1.squarespace.com/static/5992c2c7a803bb8283297efe/t/59c803110abd04d34ca9a1f0/1530629279239/" /></td>
<td style="width: 422px; height: 67px;">
<h1 style="text-align: left;">A Brief Tour of Decorators</h1>
<p><a href="https://colab.research.google.com/github/KenzieAcademy/python-notebooks/blob/master/demo_decorators.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left" width="188" height="32" /> </a></p>
</td>
</tr>
</tbody>
</table>

Decorators are part of Python that involves _Functional Programming_.

A Decorator gives a function **new behavior without changing the function itself**.

Wikipedia:
> A design pattern that allows behavior to be added to an existing object, dynamically.

In Python this means that a decorator is code that can alter a "callable". How is this enabled? By way of _function closures_.

### A brief look at function closures
You may think that _closures_ sound esoteric or mysterious but they are not really that bad. In Python, a **closure** is a function that is returned by another function and makes use of a variable from its enclosing scope.

In [ ]:
def add_x(x):
    # ------------ inner function starts here
    def adder(num):
        # adder is a closure
        # x is a free variable
        return x + num
    # ------------ inner function ends here
    return adder

# We are creating NEW custom function objects.
add_5 = add_x(5) # this is the adder() function, with number 5 enclosed within.
add_6 = add_x(6) # this is the adder() function, with number 6 enclosed within.

# They are two separate, brand new functions based on the original adder() function.
print(add_5)
print(add_6)

In [ ]:
# We have created a function closure!
# Specific job is to "add 5 to something" or "add 6 to something"
print(add_5(10))
print(add_6(10))

Examine the function above. There is an inner function, and an outer function. The inner function object is returned by the outer function.

The inner function `adder` is said to be _closed over_ -- it is a **closure**. In this example, since `x` has been defined outside the scope of `adder`, it is called a _free variable_.

Functions can return new functions. The inner function is a closure. However, the inner function relies on `x` to do its job. The closure will include this free variable `x`. Think of a closure as a package of a function object, plus any free variables it depends on.

### What is a **callable** in Python?
A callable is anything that can be _called_.  DOH.


In [ ]:
class Foo:
    def __call__(self):
        print('I have been called to a higher purpose')

foo_instance = Foo()

foo_instance() # this is calling the __call__ method

foo_instance.__call__()  # This is also calling the __call__ method!

## It's decorator time!

### Here's one that logs a function entry/exit
Please make sure you understand this example.  
1. Accepts an input function object named `func`
2. Returns a function object named `wrapper`
3. When `wrapper` is invoked, it does new things:
 - Print out a message
 - Invokes (calls) the original function object
 - Print out another message

In [ ]:
def verbosify(func):
    def wrapper():
        print("I'm about to call " + func.__name__)
        result = func()
        print("I'm done with " + func.__name__)
        return result
    return wrapper
        

A decorator is only useful when it is applied to a function.  Let's try it!  But first, let's see the original function in action

In [ ]:
def hello():
    print("Hello, all!")

In [ ]:
# Verify the original function
hello()

# What is the name of the original function?
hello.__name__

Now let's redefine the original function.  Python is a functional language!

In [ ]:
hello = verbosify(hello) # this REPLACES the original hello with a verbosified hello
hello()
hello.__name__

Yes, you saw that right. It is **not** the original function name anymore! But that's not the only beautiful thing. We now have **syntactic sugar** for this procedure

You may ask, what in the world is "Syntactic Sugar"?

> Syntactic sugar is syntax within a programming language that is designed to make things easier to read or to express. It makes the language "sweeter" for human use: things can be expressed more clearly, more concisely, or in an alternative style that some may prefer.

In [ ]:
# Place syntactic sugar decorator directly above the function
@verbosify
def greet():
    print('Good day to you!')

In [ ]:
# Call it!
greet()

### Houston we have a problem
What if the function that we are decorating accepts parameters? How do we pass parameters to a decorated function?

In [ ]:
# Example
@verbosify
def add(x, y):
    return x + y

# I sense impending doom...
add(5, 3)

### Oh drat. Tsk Tsk.
We did not define any parameters for our `wrapper` function. Ok, that is simple enough to fix...just **add parameters**! Here is a perfect use for `*args` and `**kwargs`. They allow the `wrapper` function to accept an arbitrary number of arguments, and to be passed in to the original function.

In [ ]:
# Our decorator function...revised to add args and kwargs
def verbosify(func):
    def wrapper(*args, **kwargs):
        print("I'm about to call " + func.__name__)
        result = func(*args, **kwargs)
        print("I'm done with " + func.__name__)
        return result
    return wrapper


In [ ]:
# Let's try again to pass arguments

@verbosify
def add(x, y):
    return x + y

# Smooth Sailing
add(5, 3)

In [ ]:
@verbosify
def goodbye():
    print('BUH BYE')
    
goodbye()

### Here is a lovely decorator template
Here is a generic template for a decorator that accepts arbitrary arguments

In [ ]:
# Version 1
def decorator(func_to_decorate):
    def wrapper(*args, **kwargs):
        # Do something before calling func_to_decorate
        result = func_to_decorate(*args, **kwargs)
        # Do something after calling func_to_decorate
        return result
    wrapper.__doc__ = func_to_decorate.__doc__
    wrapper.__name__ = func_to_decorate.__name__
    return wrapper

@decorator
def say_hi():
  print('hi')

print(say_hi.__name__)  # name is preserved by overriding wrapper.__name__

In [ ]:
# Version 2
from functools import wraps
def decorator(func_to_decorate):
    @wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        # Do something before calling func_to_decorate
        result = func_to_decorate(*args, **kwargs)
        # Do something after calling func_to_decorate
        return result
    return wrapper

@decorator
def say_hi():
  print('hi')

print(say_hi.__name__)  # name is preserved by functools.wraps()

You may have noticed the lines with `__doc__` and `__name__`. what is going on there?

A well-behaved decorator should have the `__doc__` and `__name__` attributes of its inner wrapper updated to reflect those of the original decorated function so that it is "friendly to introspection" -- it should not change the names of functions. This is important for things like serialization of objects.

### Other ways to create decorators
Remember that any _callable_ can be used to decorate a function. This includes not only functions (which are naturally callable) but also class objects. To put it simply: a decorator is a callable that accepts a callable and returns a callable.

In [ ]:
class deco_class:
    def __init__(self, func):
        self.func = func
    # by implementing the __call__ magic method, this class is now `callable`
    def __call__(self, *args, **kwargs):
        print('deco class before invoking its func')
        result = self.func(*args, **kwargs)
        print('deco class after invoking its func')

In [ ]:
@deco_class
def my_other_func():
    print("I feel like I've been OTHERED")
    
my_other_func()

### Decorator examples
This decorator works by storing the time just before the function starts running (at the line marked # 1) and just after the function finishes (at # 2). The time the function takes is then the difference between the two (at # 3).

In [ ]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.time()    # 1
        value = func(*args, **kwargs)
        end_time = time.time()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [ ]:
waste_some_time(1)

In [ ]:
waste_some_time(999)

Note: This `@timer` decorator is great if you just want to get an idea about the runtime of your functions. If you want to do more precise measurements of code, you should instead consider the timeit module in the standard library. It temporarily disables garbage collection and runs multiple trials to strip out noise from quick function calls.

### Maintaining state within a decorator

In [ ]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__}")
        return func(*args, **kwargs)
    # This adds a new attribute variable to the `wrapper_count_calls` function!
    # The state of this counter is preserved between successive calls.
    wrapper_count_calls.num_calls = 0  
    
    return wrapper_count_calls

@count_calls
def say_ticktock():
    print("Tick Tock!")

say_ticktock()
say_ticktock()
for _ in range(1, 10):
    say_ticktock()

## Conclusions
 - Decorators are cool. Functional programming is cool.
 - You will use decorators in Django.
 - We have only scratched the surface here!